In [1]:
import matplotlib
matplotlib.use('Agg') #  disable interactive matplotlib to save RAM

mice = ['1_5']
 #min
exclude_sessions = {
    # # Example
    # # '6_2': [
    # #     'data/spikeinterface-0_98_2/Behavior/6_2/20231208/session_1',
    # # ],
}
nsx = 'ns4'

import datetime
import glob
import numpy as np
import os
import pandas as pd
import spikeinterface.core as sc
import spikeinterface.curation as scu
import spikeinterface.extractors as se
import spikeinterface.sorters as ss
import sys 

from tqdm.auto import tqdm

sys.path.append('src')

from utils import *

today = '20240120'#datetime.datetime.today().strftime('%Y%m%d')
sampling_frequency = n_ms_per_s * (10 if nsx == 'ns4' else 30 if nsx == 'ns6' else None)
data_root = 'data'
si_folder = f'{data_root}{os.sep}spikeinterface-0_98_2'
sorted_folder = f'{data_root}{os.sep}sorted{os.sep}{today}'

print(f'Saving to {sorted_folder}')

Saving to data/sorted/20240120


In [2]:
recording_session_paths = {}
for mouse in mice:
    recording_session_paths[mouse] = sorted([
        session for session in glob.glob(f'{si_folder}{os.sep}LongTerm-{nsx}{os.sep}{mouse}{os.sep}**{os.sep}**') 
            if session not in exclude_sessions.get(mouse, [])
    ])
recording_session_paths

{'1_5': ['data/spikeinterface-0_98_2/LongTerm-ns4/1_5/20230707/063027sebs15001',
  'data/spikeinterface-0_98_2/LongTerm-ns4/1_5/20230714/datafile002',
  'data/spikeinterface-0_98_2/LongTerm-ns4/1_5/20230721/datafile001',
  'data/spikeinterface-0_98_2/LongTerm-ns4/1_5/20230727/20230727_SEBS1-5_001',
  'data/spikeinterface-0_98_2/LongTerm-ns4/1_5/20230821/datafile001',
  'data/spikeinterface-0_98_2/LongTerm-ns4/1_5/20230831/datafile',
  'data/spikeinterface-0_98_2/LongTerm-ns4/1_5/20230921/datafile',
  'data/spikeinterface-0_98_2/LongTerm-ns4/1_5/20231006/datafile',
  'data/spikeinterface-0_98_2/LongTerm-ns4/1_5/20231011/datafile',
  'data/spikeinterface-0_98_2/LongTerm-ns4/1_5/20231018/datafile',
  'data/spikeinterface-0_98_2/LongTerm-ns4/1_5/20231027/datafile',
  'data/spikeinterface-0_98_2/LongTerm-ns4/1_5/20231102/datafile',
  'data/spikeinterface-0_98_2/LongTerm-ns4/1_5/20231122/datafile007',
  'data/spikeinterface-0_98_2/LongTerm-ns4/1_5/20231207/datafile',
  'data/spikeinterface-0

In [4]:
for mouse in (pbar := tqdm(mice)):
    mouse_sorted_folder = f'{sorted_folder}{os.sep}{mouse}'
    pbar.set_description(mouse)

    mouse_processed_traces_path = f'{mouse_sorted_folder}{os.sep}traces_processed'
    mouse_recording_si_path = f'{mouse_sorted_folder}{os.sep}processed'
    mouse_sorting_si_path = f'{mouse_sorted_folder}{os.sep}sorting-by-group'
    mouse_waveforms_si_path = f'{mouse_sorted_folder}{os.sep}waveforms-by-group'
    mouse_units_si_path = f'{mouse_sorted_folder}{os.sep}units-by-group'
    mouse_sessions_file = f'{mouse_sorted_folder}{os.sep}sessions.csv'
    os.makedirs(mouse_processed_traces_path, exist_ok=True)

    if True:# not os.path.isfile(mouse_sessions_file):
        mouse_traces, mouse_sessions = [], []
        cumulative_samples = 0
        for session_index, session_path in enumerate(recording_session_paths[mouse]):
            session_date = session_path.split(os.sep)[-2]
            session_segments = longterm_segments[mouse][session_date]
            session_trace_plot_file = f'{mouse_processed_traces_path}{os.sep}[{session_index}]{session_date}.png'
            pbar.set_description(f'{mouse} -> [{session_index+1} / {len(recording_session_paths[mouse])}]{session_date} -> {session_segments}')

            recording = sc.load_extractor(f'{session_path}{os.sep}raw')
            full_traces = recording.get_traces()
            
            traces = np.vstack([
                full_traces[segment_start*sampling_frequency*n_s_per_min : segment_end*sampling_frequency*n_s_per_min] 
                    for segment_start, segment_end in session_segments
            ])
            if not os.path.isfile(session_trace_plot_file):
                plot_traces(traces.T, sampling_frequency, blackrock_channel_indices, title=f'{mouse} -> {session_date} -> {session_segments}', savepath=session_trace_plot_file, trace_gap=150)

    #         mouse_traces.append(traces)
    #         mouse_sessions.append({
    #             'mouse': mouse,
    #             'date': session_date,
    #             'session_path': session_path,
    #             'session_start': cumulative_samples,
    #             'sampling_frequency': sampling_frequency,
    #             'session_length': traces.shape[0],
    #         })
    #         cumulative_samples += traces.shape[0] 
    #     mouse_traces = np.vstack(mouse_traces)

    #     recording = se.NumpyRecording(traces_list=mouse_traces, sampling_frequency=sampling_frequency)
    #     # recording_processed = preprocess_recording(recording, steps=['bp', 'cmr', 'clip'])
    #     recording_processed = preprocess_recording(recording)
        
    #     os.makedirs(mouse_sorted_folder, exist_ok=True)
    #     multi_shank_probe = create_probe(blackrock_channel_indices, f'{mouse_sorted_folder}{os.sep}probe.png')
    #     recording_processed.set_probe(multi_shank_probe, in_place=True)

    #     shutil.rmtree(mouse_recording_si_path, ignore_errors=True)
    #     recording_processed.save(folder=mouse_recording_si_path, memory=memory_limit)

    #     mouse_sessions = pd.json_normalize(mouse_sessions)
    #     mouse_sessions.to_csv(mouse_sessions_file, index=False)
    # pbar.set_description(f'{mouse} preprocessed')

    # if not os.path.isfile(f'{mouse_sorting_si_path}{os.sep}sorter_output{os.sep}firings.npz'):
    #     recording_processed = sc.load_extractor(mouse_recording_si_path)
    #     probegroup = create_probegroup(blackrock_channel_indices)
    #     recording_processed.set_probegroup(probegroup, in_place=True)

    #     sorting = ss.run_sorter_by_property(
    #         sorter_name='mountainsort4',
    #         recording=recording_processed,
    #         grouping_property='group',
    #         working_folder=mouse_sorting_si_path,
    #         mode_if_folder_exists='overwrite',
    #         **sorter_parameters,
    #     )
    #     os.makedirs(f'{mouse_sorting_si_path}{os.sep}sorter_output', exist_ok=True)
    #     se.NpzSortingExtractor.write_sorting(sorting, f'{mouse_sorting_si_path}{os.sep}sorter_output{os.sep}firings.npz')
    # pbar.set_description(f'{mouse} sorted')

    # if not os.path.isfile(f'{mouse_waveforms_si_path}{os.sep}templates_average.npy'):
    #     recording_processed = sc.load_extractor(mouse_recording_si_path)
    #     sorting = se.NpzSortingExtractor(f'{mouse_sorting_si_path}{os.sep}sorter_output{os.sep}firings.npz')
    #     sorting = scu.remove_excess_spikes(sorting, recording_processed) # spikeinterface https://github.com/SpikeInterface/spikeinterface/pull/1378
    #     sc.extract_waveforms(
    #         recording_processed, sorting, 
    #         folder=mouse_waveforms_si_path,
    #         ms_before=ms_before, ms_after=ms_after, max_spikes_per_unit=None,
    #         return_scaled=False,
    #         overwrite=True,
    #         use_relative_path=True,
    #     )
    # pbar.set_description(f'{mouse} extracted')

    # recording_processed, sorting, waveform_extractor, extremum_channels, mouse_sessions = read_sorted_results(mouse_sorted_folder, read_sessions=True, by_group=True)

    # os.makedirs(mouse_units_si_path, exist_ok=True)
    # for unit_id in sorting.unit_ids:
    #     pbar.set_description(f'{mouse} Plotting [unit {unit_id} / {len(sorting.unit_ids)}]')
    #     unit_plot_file = f'{mouse_units_si_path}{os.sep}{unit_id}.png'
    #     if not os.path.isfile(unit_plot_file):
    #         plot_unit(waveform_extractor, extremum_channels, sorting, unit_id, blackrock_channel_indices, initdate=surgery_dates[mouse], savepath=unit_plot_file, sessions=mouse_sessions)

  0%|          | 0/1 [00:00<?, ?it/s]

KeyboardInterrupt: 